In [1]:
# 相关包与其路径初配置
import sys
from pathlib import Path

# 获取当前文件的路径
current_path = Path().absolute()
# 添加项目根目录（包含 docx_parser 和 llm_structuring 的目录）到系统路径
root_path = current_path.parent.parent
sys.path.append(str(root_path))

from docx_parser_package.docx_parser.pipeline import DocxParserPipeline
from docx_parser_package.docx_parser._03_element_extractor import ElementType
from doc_structurer_package.doc_strucuturer._03_tree_builder import TreeBuilder
from doc_structurer_package.doc_strucuturer.doc_tree_retriever import DocTreeRetriever
from doc_structurer_package.enrich_doc_structure._01_node_pre_screener import HeadNodePreScreener
from doc_structurer_package.enrich_doc_structure._02_LLM_analyzer import LLMAnalyzer


In [2]:
# 文档路径
#doc_path = "C:/Users/huiwa/Downloads/文本分析测试/CaseTest/case3_北京铁运投标人须知.docx"
#doc_path = "C:/Users/huiwa/Downloads/文本分析测试/CaseTest/case3st_北京铁运投标人须知.docx"
#doc_path = "C:/Users/huiwa/Downloads/文本分析测试/CaseTest/case 9 _样式测试.docx"
doc_path = "C:/Users/huiwa/Downloads/文本分析测试/CaseTest/case8：招标文件-第1包：一级压榨花生油.docx"
#doc_path = "C:/Users/huiwa/Documents/_CursorChat_Extra/Langchain_Jupyter/.LoRA微调/招标文件_微调库/WORD/[2].docx"
#doc_path = "C:/Users/huiwa/Downloads/文本分析测试/CaseTest/Case6_中国区零食包框架供应商采购项目招采文件.docx"

#### <font color="red">1. 获取Document Elements</font>

In [3]:
# 获取Document Elements
pipeline = DocxParserPipeline(doc_path)
elements = pipeline.process()

2024-12-31 00:25:39,254 - docx_parser_package.docx_parser._01_xml_loader - INFO - 从 C:\Users\huiwa\Downloads\文本分析测试\CaseTest\case8：招标文件-第1包：一级压榨花生油.docx 成功提取 raw XML content
2024-12-31 00:25:39,254 - docx_parser_package.docx_parser.pipeline - INFO - 成功加载文档: C:\Users\huiwa\Downloads\文本分析测试\CaseTest\case8：招标文件-第1包：一级压榨花生油.docx
2024-12-31 00:25:39,275 - docx_parser_package.docx_parser._02_xml_parser - INFO - Successfully parsed XML content
2024-12-31 00:25:39,275 - docx_parser_package.docx_parser._02_xml_parser - INFO - DocxXMLParser initialized successfully
2024-12-31 00:25:39,276 - docx_parser_package.docx_parser.pipeline - INFO - 成功解析XML结构
2024-12-31 00:25:39,288 - docx_parser_package.docx_parser._03_element_extractor - INFO - Starting element extraction...
2024-12-31 00:25:39,289 - docx_parser_package.docx_parser._03_element_extractor - DEBUG - Found 928 total elements
2024-12-31 00:25:39,289 - docx_parser_package.docx_parser._03_element_extractor - DEBUG - Processing element 1, type:

#### <font color="red">2. 构建文档树</font>

In [4]:
# 构建文档树
tree_builder = TreeBuilder(elements)
doc_structure = tree_builder.build_to_level(target_level=2)

In [5]:
nodes = doc_structure.doc_nodes
print(f"{doc_structure.doc_tree.node_id}, root length: {doc_structure.doc_tree.branch_length}")
for node in nodes:
    print(f"{node.node_id}, {node.node_type}, level: {node.level}, " 
          f"node_length: {node.node_length}, branch_length: {node.branch_length}, "
          f"#nodes: {node.ttl_nodes_in_branch}, content: {node.element.content[:20]}")


-1, root length: 36112
1, content_node, level: None, node_length: 30, branch_length: None, #nodes: None, content: 北京京铁运恒采购供应站有限公司 2024
2, content_node, level: None, node_length: 4, branch_length: None, #nodes: None, content: 招标文件
3, content_node, level: None, node_length: 54, branch_length: None, #nodes: None, content:         项目名称：北京京铁运恒采
4, content_node, level: None, node_length: 12, branch_length: None, #nodes: None, content: 项目编号：DLXM-2024-148-0
5, content_node, level: None, node_length: 39, branch_length: None, #nodes: None, content: 招 标 人：北京京铁运恒采购供应站有限公
6, content_node, level: None, node_length: 10, branch_length: None, #nodes: None, content:                 2024
7, content_node, level: None, node_length: 6, branch_length: None, #nodes: None, content:                     
8, content_node, level: None, node_length: 10, branch_length: None, #nodes: None, content: 第一章  招标公告
9, content_node, level: None, node_length: 10, branch_length: None, #nodes: None, content: 第二章  招标需求
10, conte

In [6]:
# 添加文档树检索器
doc_tree_retriever = DocTreeRetriever(doc_structure.doc_tree)

In [7]:
# 获取文档所有内容
doc_tree_retriever.get_all_content()

['北京京铁运恒采购供应站有限公司 2024 年端午节物资采购项目',
 '招标文件',
 '        项目名称：北京京铁运恒采购供应站有限公司 2024 年端午节 物资采购项目（第一包：一级压榨花生油）',
 '项目编号：DLXM-2024-148-01',
 '招 标 人：北京京铁运恒采购供应站有限公司 代理机构：北京中外建工程管理有限公司',
 '                2024 年 3 月',
 '                    目    录',
 '第一章  招标公告',
 '第二章  招标需求',
 '第三章  投标人须知',
 '前附表',
 '一、总则',
 '二、招标文件',
 '三、投标文件的编制',
 '四、开标',
 '五、评标',
 '六、定标',
 '七、合同授予',
 '八、履约保证金',
 '九、招标代理服务费的收取',
 '第四章  评标办法及评分标准',
 '附件一：资格审查表',
 '附件二：符合性审查表',
 '附件三：评分标准',
 '第五章  合同条款及格式',
 '第六章  投标文件格式',
 '附件 1 投标函',
 '附件 2 投标保证书',
 '附件 3 无违法记录声明',
 '附件 4 法定代表人授权书',
 '附件 5 投标人情况简介',
 '附件 6 投标人 2021 年 1 月 1 日起至投标截止日止同类产品销售业绩表',
 '附件 7 投标一览表',
 '附件 8 分项报价表',
 '附件 9 投标方案',
 '附件 10 投标人资格声明',
 '附件 11 资格证明文件',
 '附件 12 与投标人存在“关系”的其他法人单位信息',
 '附件 13 投标人认为必要的其他证明文件',
 '附件 14 技术规格偏离表',
 '附件 15 商务条款偏离表',
 '附件 16 拟投入本项目的人员配备表',
 '附件 17 中标服务费承诺书',
 '附件 18 关于投标保证金的声明',
 '附件 19 开标会声明书',
 '第一章 招标公告',
 '根据《中华人民共和国招标投标法》、《中华人民共和国招标投标法实施条 例》等规定， 北京中外建工程管理有限公司受招标人委托，就北京京铁运恒采购 供应站有限公司 2024 年端午节物资采购项目（第一包：一级压榨花生油）进行  国内公开招标采购，欢迎合

In [8]:
# 打印所有标题节点
doc_tree_retriever.print_list_of_all_heading_nodes_by_tree()

root （51 子nodes，其中 6 子标题nodes）
  ├─ 46 第一章 招标公告  （48 子nodes, 其中 0 子标题nodes）
  ├─ 95 第二章 招标需求  （28 子nodes, 其中 0 子标题nodes）
  ├─ 124 第三章 投标人须知  （10 子nodes, 其中 10 子标题nodes）
  |    |  ├─ 125 前附表  （4 子nodes, 其中 0 子标题nodes）
  |    |  ├─ 130 一、总则  （28 子nodes, 其中 0 子标题nodes）
  |    |  ├─ 159 二、招标文件  （15 子nodes, 其中 0 子标题nodes）
  |    |  ├─ 175 三、投标文件的编制  （80 子nodes, 其中 0 子标题nodes）
  |    |  ├─ 256 四、开标  （10 子nodes, 其中 0 子标题nodes）
  |    |  ├─ 267 五、评标  （26 子nodes, 其中 0 子标题nodes）
  |    |  ├─ 294 六、定标  （3 子nodes, 其中 0 子标题nodes）
  |    |  ├─ 298 七、合同授予  （1 子nodes, 其中 0 子标题nodes）
  |    |  ├─ 300 八、履约保证金  （1 子nodes, 其中 0 子标题nodes）
  |    |  └─ 302 九、招标代理服务费的收取  （5 子nodes, 其中 0 子标题nodes）
  ├─ 308 第四章 评标办法及评分标准  （46 子nodes, 其中 3 子标题nodes）
  |    |  ├─ 352 附件一：资格审查表  （2 子nodes, 其中 0 子标题nodes）
  |    |  ├─ 355 附件二：符合性审查表  （1 子nodes, 其中 0 子标题nodes）
  |    |  └─ 357 附件三：评分标准  （8 子nodes, 其中 0 子标题nodes）
  ├─ 366 第五章 合同条款及格式  （134 子nodes, 其中 0 子标题nodes）
  └─ 501 第六章 投标文件格式  （32 子nodes, 其中 19 子标题nodes）
     

In [9]:
# 获取指定层级的内容
doc_tree_retriever.get_headings_by_level(level=1)

['第一章 招标公告',
 '第二章 招标需求',
 '第三章 投标人须知',
 '第四章 评标办法及评分标准',
 '第五章 合同条款及格式',
 '第六章 投标文件格式']

总结：构建的文档树 有doc_structure封装， 里面有doc_structure.doc_nodes 和 doc_structure.doc_tree 两种可访问数据形式。


#### <font color="red">3. **文档树 分析**</font>


In [10]:
# 所有标题节点
title_nodes = [node for node in doc_structure.doc_nodes if node.element.is_heading]
for node in title_nodes:

    indent = "    " * node.level
    print(node.node_id, node.enrich_pre_screened_result, indent, node.element.content,node.ttl_nodes_in_branch, node.branch_length,)



46 None      第一章 招标公告 48 2306
95 None      第二章 招标需求 28 1620
124 None      第三章 投标人须知 183 10008
125 None          前附表 4 2731
130 None          一、总则 28 1193
159 None          二、招标文件 15 629
175 None          三、投标文件的编制 80 3219
256 None          四、开标 10 375
267 None          五、评标 26 1187
294 None          六、定标 3 163
298 None          七、合同授予 1 73
300 None          八、履约保证金 1 45
302 None          九、招标代理服务费的收取 5 327
308 None      第四章 评标办法及评分标准 57 7023
352 None          附件一：资格审查表 2 1022
355 None          附件二：符合性审查表 1 488
357 None          附件三：评分标准 8 3397
366 None      第五章 合同条款及格式 134 7682
501 None      第六章 投标文件格式 153 6832
515 None          附件 1 投标函 16 485
532 None          附件 2 投标保证书 7 217
540 None          附件 3 无违法记录声明 7 82
548 None          附件 4 法定代表人授权书 7 231
556 None          附件 5 投标人情况简介 2 1299
559 None          附件 6 投标人 2021 年 1 月 1 日起至投标截止日止同类产品销售业绩表 2 154
562 None          附件 7 投标一览表 7 413
570 None          附件 8 分项报价表 8 369
579 None          附件 9 投标方案 9 187
589 None          附件 10 投标人资格声明

In [11]:
# 文档树最大层级
max_level = max((node.level for node in doc_structure.doc_nodes if node.level is not None), default=None)
print(max_level)

2


##### <font color=orange> 3.1 评估哪些节点还可能是标题？ 正则式反向排除 </font>

In [12]:
# 从文本特征初步评估文本为标题的可能性， 使用了HeadNodePreScreener的pre_screen_nodes()方法
doc_nodes = doc_structure.doc_nodes
head_node_pre_screener = HeadNodePreScreener(doc_nodes)
head_node_pre_screener.pre_screen_nodes()


In [13]:
# Print 展示 所有Nodes的 pre_screening 的结果
print("="*50)
content_count = sum(1 for node in doc_nodes if node.enrich_pre_screened_result == "content")
undetermined_count = sum(1 for node in doc_nodes if node.enrich_pre_screened_result == "undetermined")
print(f"以下为{len(doc_nodes)} 个样本的测试结果, 确定非标题个数: {content_count}, 可能是标题个数: {undetermined_count}")
print("="*50)
for node in doc_nodes:
    print(node.node_id, node.enrich_pre_screened_result, node.element.content)

以下为654 个样本的测试结果, 确定非标题个数: 0, 可能是标题个数: 0
1 [待定]   北京京铁运恒采购供应站有限公司 2024 年端午节物资采购项目
2 [待定]   招标文件
3 [正文]         项目名称：北京京铁运恒采购供应站有限公司 2024 年端午节 物资采购项目（第一包：一级压榨花生油）
4 [待定]   项目编号：DLXM-2024-148-01
5 [待定]   招 标 人：北京京铁运恒采购供应站有限公司 代理机构：北京中外建工程管理有限公司
6 [待定]                   2024 年 3 月
7 [目录]                     目    录
8 [目录] 第一章  招标公告
9 [目录] 第二章  招标需求
10 [目录] 第三章  投标人须知
11 [目录] 前附表
12 [目录] 一、总则
13 [目录] 二、招标文件
14 [目录] 三、投标文件的编制
15 [目录] 四、开标
16 [目录] 五、评标
17 [目录] 六、定标
18 [目录] 七、合同授予
19 [目录] 八、履约保证金
20 [目录] 九、招标代理服务费的收取
21 [目录] 第四章  评标办法及评分标准
22 [目录] 附件一：资格审查表
23 [目录] 附件二：符合性审查表
24 [目录] 附件三：评分标准
25 [目录] 第五章  合同条款及格式
26 [目录] 第六章  投标文件格式
27 [目录] 附件 1 投标函
28 [目录] 附件 2 投标保证书
29 [目录] 附件 3 无违法记录声明
30 [目录] 附件 4 法定代表人授权书
31 [目录] 附件 5 投标人情况简介
32 [目录] 附件 6 投标人 2021 年 1 月 1 日起至投标截止日止同类产品销售业绩表
33 [目录] 附件 7 投标一览表
34 [目录] 附件 8 分项报价表
35 [目录] 附件 9 投标方案
36 [目录] 附件 10 投标人资格声明
37 [目录] 附件 11 资格证明文件
38 [目录] 附件 12 与投标人存在“关系”的其他法人单位信息
39 [目录] 附件 13 投标人认为必要的其他证明文件
40 [目录] 附件 14 技术规格偏离表
41 [目录] 附件 15 商务条款偏离表
42 [目录] 附件 16

##### <font color=orange>挑选末端标题节点 </font>

In [14]:
#找到末端标题节点（有的chapter, 有的是section, 有的是小节）
branch_end_nodes = [node for node in doc_structure.doc_nodes 
                    if node.element.is_heading 
                    and node.children is not None
                    and not any(child.node_type == "title_node" for child in node.children)  
                    ]

for i, node in enumerate(branch_end_nodes):
    print(i, node.node_id, node.enrich_pre_screened_result, node.element.content)

0 46 [章] 第一章 招标公告
1 95 [章] 第二章 招标需求
2 125 [附件] 前附表
3 130 [节] 一、总则
4 159 [节] 二、招标文件
5 175 [节] 三、投标文件的编制
6 256 [节] 四、开标
7 267 [节] 五、评标
8 294 [节] 六、定标
9 298 [节] 七、合同授予
10 300 [节] 八、履约保证金
11 302 [节] 九、招标代理服务费的收取
12 352 [附件] 附件一：资格审查表
13 355 [附件] 附件二：符合性审查表
14 357 [附件] 附件三：评分标准
15 366 [章] 第五章 合同条款及格式
16 515 [附件] 附件 1 投标函
17 532 [附件] 附件 2 投标保证书
18 540 [附件] 附件 3 无违法记录声明
19 548 [附件] 附件 4 法定代表人授权书
20 556 [附件] 附件 5 投标人情况简介
21 559 [附件] 附件 6 投标人 2021 年 1 月 1 日起至投标截止日止同类产品销售业绩表
22 562 [附件] 附件 7 投标一览表
23 570 [附件] 附件 8 分项报价表
24 579 [附件] 附件 9 投标方案
25 589 [附件] 附件 10 投标人资格声明
26 596 [附件] 附件 11 资格证明文件
27 607 [附件] 附件 12 与投标人存在“关系”的其他法人单位信息
28 611 [附件] 附件 13 投标人认为必要的其他证明文件
29 612 [附件] 附件 14 技术规格偏离表
30 618 [附件] 附件 15 商务条款偏离表
31 624 [附件] 附件 16 拟投入本项目的人员配备表
32 630 [附件] 附件 17 中标服务费承诺书
33 636 [附件] 附件 18 关于投标保证金的声明
34 646 [附件] 附件 19 开标会声明书


In [15]:
#找到末端标题节点（有的chapter, 有的是section, 有的是小节）
branch_end_nodes = [node for node in doc_structure.doc_nodes 
                    if node.element.is_heading 
                    and node.children is not None
                    and not any(child.enrich_pre_screened_result == "[章]" 
                                or child.enrich_pre_screened_result == "[节]" 
                                or child.enrich_pre_screened_result == "[小节]" for child in node.children)  
                    ]

for i, node in enumerate(branch_end_nodes):
    print(i, node.node_id, node.enrich_pre_screened_result, node.element.content)



0 46 [章] 第一章 招标公告
1 95 [章] 第二章 招标需求
2 125 [附件] 前附表
3 130 [节] 一、总则
4 159 [节] 二、招标文件
5 175 [节] 三、投标文件的编制
6 256 [节] 四、开标
7 267 [节] 五、评标
8 294 [节] 六、定标
9 298 [节] 七、合同授予
10 300 [节] 八、履约保证金
11 302 [节] 九、招标代理服务费的收取
12 308 [章] 第四章 评标办法及评分标准
13 352 [附件] 附件一：资格审查表
14 355 [附件] 附件二：符合性审查表
15 357 [附件] 附件三：评分标准
16 366 [章] 第五章 合同条款及格式
17 501 [章] 第六章 投标文件格式
18 515 [附件] 附件 1 投标函
19 532 [附件] 附件 2 投标保证书
20 540 [附件] 附件 3 无违法记录声明
21 548 [附件] 附件 4 法定代表人授权书
22 556 [附件] 附件 5 投标人情况简介
23 559 [附件] 附件 6 投标人 2021 年 1 月 1 日起至投标截止日止同类产品销售业绩表
24 562 [附件] 附件 7 投标一览表
25 570 [附件] 附件 8 分项报价表
26 579 [附件] 附件 9 投标方案
27 589 [附件] 附件 10 投标人资格声明
28 596 [附件] 附件 11 资格证明文件
29 607 [附件] 附件 12 与投标人存在“关系”的其他法人单位信息
30 611 [附件] 附件 13 投标人认为必要的其他证明文件
31 612 [附件] 附件 14 技术规格偏离表
32 618 [附件] 附件 15 商务条款偏离表
33 624 [附件] 附件 16 拟投入本项目的人员配备表
34 630 [附件] 附件 17 中标服务费承诺书
35 636 [附件] 附件 18 关于投标保证金的声明
36 646 [附件] 附件 19 开标会声明书


In [16]:
# 添加筛选条件：如节点长度>2000tokens，节点子元素>3, 节点内容为非附件； 待优化，目前先不考虑
selected_branch_end_nodes = [node for node in branch_end_nodes 
                   if node.branch_length > 1000
                   and node.ttl_nodes_in_branch > 3
                   and node.enrich_pre_screened_result != "[附件]"]
for node in selected_branch_end_nodes:
    print(node.node_id, node.enrich_pre_screened_result, node.element.content)

46 [章] 第一章 招标公告
95 [章] 第二章 招标需求
130 [节] 一、总则
175 [节] 三、投标文件的编制
267 [节] 五、评标
308 [章] 第四章 评标办法及评分标准
366 [章] 第五章 合同条款及格式
501 [章] 第六章 投标文件格式


In [17]:
#找到末端标题节点（有的chapter, 有的是section, 有的是小节）
mid_branch_nodes = [node for node in doc_structure.doc_nodes 
                    if node.element.is_heading 
                    and node.children is not None
                    and any(child.node_type == "content_node" for child in node.children) and any(child.node_type == "title_node" for child in node.children)  
                    ]

for i, node in enumerate(mid_branch_nodes):
    print(i, node.node_id, node.enrich_pre_screened_result, node.element.content)

0 308 [章] 第四章 评标办法及评分标准
1 501 [章] 第六章 投标文件格式


##### <font color = orange> 按特定格式 组织 章节文本  （考虑单独函数）</font>

In [18]:
# 计算文本的token数量
import tiktoken
def count_tokens(text: str) -> int:
    """计算文本的token数量"""
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    return len(encoding.encode(text))

In [19]:
# 为节点生成Parts用于大模型分析，每个Part都是特定格式的文本
parts = []

for node in branch_end_nodes:
    # 构建文本片段
    part = {}
    text_part = []
    # 添加标题部分
    heading_type = "章" if node.element.heading_level == 1 else "节" if node.element.heading_level == 2 else "小节"
    text_part.append(f"章节编号：{node.node_id}")
    text_part.append(f"章节标题：{node.element.content}")
    
    # 添加正文部分
    text_part.append("正文内容：")
    
    # 遍历子节点，添加内容
    for child in node.children:
        if child.enrich_pre_screened_result == "[附件]":
            text_part.append(f"附件部分：{child.element.content} （这里只展示附件名称）")
        elif child.element.element_type == ElementType.TABLE:
            text_part.append(f"表格：{child.element.content}")
        else:
            text_part.append(f"段落：{child.element.content}")
    # 将所有部分用换行符连接成一个字符串
    final_text = "\n".join(text_part)
    part['content'] = final_text
    part['type'] = node.enrich_pre_screened_result
    part['part_length'] = count_tokens(final_text)

    parts.append(part)



In [20]:
# 打印第一个片段查看效果
print(parts[18]['type'])
print(parts[18]['part_length'])
print(parts[18]['content'])

[附件]
578
章节编号：515
章节标题：附件 1 投标函
正文内容：
段落：致：招标人
段落：根据贵方为                 项目招标采购货物及服务的招标要求，签字代 表          （全名、职务）经正式授权并代表投标人                  （投 标人名称、地址）提交下述文件正本 一 份和副本       份及电子版      份。
段落：1、开标一览表
段落：2、分项报价表
段落：3、投标方案
段落：4、投标须知要求提供的全部文件
段落：5、投标保证金，金额为人民币       元。 据此函，签字代表宣布同意如下：
段落：6、附投标报价表中规定的应提供和交付的货物投标总价为人民币       元。
段落：7、投标人将按招标文件的规定履行合同责任和义务。
段落：8、投标人已详细审查全部招标文件， 包括修改文件（如需要修改）以及全部 参考资料和有关附件。我们完全理解并同意放弃对这方面有不明及误解的权利。
段落：9、投标有效期为                    日历天。
段落：10、在规定的开标日期后，投标人在投标有效期内撤回投标，其投标保证金 将被贵方没收。
段落：11、投标人同意提供按照贵方可能要求的与其投标有关的一切数据或资料， 完全理解贵方不一定接受最低价格的投标或收到的任何投标。
段落：12、本投标有关的一切正式往来通讯请寄：
段落：地址：                                      邮编： 电话：                                      传真： 投标人名称（盖章）：
段落：法定代表人或被授权代表(签字或盖章):                      日期：


<font color=orange> 大模型分析，输出章节 摘要、章节内容、作用、关键行动。 </font>




In [21]:
Appendix = """
请为我分析以下附件的文本内容，提供它的简明扼要的信息，以json格式输出如下：
[{
 "章节编号": "章节编号",
 "章节标题": "章节标题",
 "附件类型": "模板|补充说明",
 "摘要": "一句话概括附件核心内容",
 "内容包括": "关键信息点列举", 
 "解读环节用途": "招标文件解读环节的作用",
 "编制环节用途": "投标文件编制环节的作用",
 "关键行动": [
   {
     "行动名称": "...",
     "时间": "...",
     "地点": "...",
     "内容": "..."
   }
 ]
}]

文本内容：
<text>
"""

general_prompt = """
请为我分析以下章节的文本内容，提供它的简明扼要的信息，以json格式输出如下：
[{
 "章节编号": "章节编号",
 "章节标题": "章节标题",
 "附件类型": "模板|补充说明",
 "摘要": "一句话概括附件核心内容",
 "内容包括": "关键信息点列举", 
 "解读环节用途": "招标文件解读环节的作用",
 "编制环节用途": "投标文件编制环节的作用",
 "关键行动": [
   {
     "行动名称": "...",
     "时间": "...",
     "地点": "...",
     "内容": "..."
   }
 ]
}]

文本内容：
<text>
"""

prompts = {
    "appendix": Appendix,
    "chapter": general_prompt,
    "section": general_prompt,
    "sub_section": general_prompt,
}

In [22]:
analyzer = LLMAnalyzer(prompts["appendix"], parts[18])


In [23]:
result = analyzer.analyze_parts()




开始分析...
```json
{
  "章节编号": "515",
  "章节标题": "附件 1 投标函",
  "附件类型": "模板",
  "摘要": "投标函模板，包含投标文件提交、保证金、合同责任等关键信息",
  "内容包括": "投标文件提交、投标保证金、合同责任、投标有效期、撤回投标的后果、提供额外数据或资料的承诺、正式通讯地址",
  "解读环节用途": "帮助招标人理解投标函的格式和内容要求，确保投标文件的完整性和合规性",
  "编制环节用途": "指导投标人如何填写和提交投标函，明确投标函中的各项承诺和责任",
  "关键行动": [
    {
      "行动名称": "提交投标文件",
      "时间": "投标截止日期前",
      "地点": "招标人指定地点",
      "内容": "提交投标函正本一份、副本若干份及电子版"
    },
    {
      "行动名称": "支付投标保证金",
      "时间": "投标截止日期前",
      "地点": "招标人指定账户",
      "内容": "支付投标保证金，金额为人民币若干元"
    },
    {
      "行动名称": "履行合同责任",
      "时间": "中标后",
      "地点": "项目实施地点",
      "内容": "按照招标文件规定履行合同责任和义务"
    },
    {
      "行动名称": "撤回投标",
      "时间": "开标后投标有效期内",
      "地点": "招标人指定地点",
      "内容": "在投标有效期内撤回投标，投标保证金将被没收"
    },
    {
      "行动名称": "提供额外数据或资料",
      "时间": "招标人要求时",
      "地点": "招标人指定地点",
      "内容": "提供与投标有关的一切数据或资料"
    }
  ]
}
```返回结果必须是列表格式，当前格式为: <class 'dict'>

process_part_async 返回值检查:
result 类型: <class 'list'>
result 内容: [{'章节编号': '515', '章节标题': '附件 1 

In [24]:
print(type(result))

<class 'list'>


In [25]:
from pprint import pprint
pprint(result)


[{'关键行动': [{'内容': '提交投标函正本一份、副本若干份及电子版',
            '地点': '招标人指定地点',
            '时间': '投标截止日期前',
            '行动名称': '提交投标文件'},
           {'内容': '支付投标保证金，金额为人民币若干元',
            '地点': '招标人指定账户',
            '时间': '投标截止日期前',
            '行动名称': '支付投标保证金'},
           {'内容': '按照招标文件规定履行合同责任和义务',
            '地点': '项目实施地点',
            '时间': '中标后',
            '行动名称': '履行合同责任'},
           {'内容': '在投标有效期内撤回投标，投标保证金将被没收',
            '地点': '招标人指定地点',
            '时间': '开标后投标有效期内',
            '行动名称': '撤回投标'},
           {'内容': '提供与投标有关的一切数据或资料',
            '地点': '招标人指定地点',
            '时间': '招标人要求时',
            '行动名称': '提供额外数据或资料'}],
  '内容包括': '投标文件提交、投标保证金、合同责任、投标有效期、撤回投标的后果、提供额外数据或资料的承诺、正式通讯地址',
  '摘要': '投标函模板，包含投标文件提交、保证金、合同责任等关键信息',
  '章节标题': '附件 1 投标函',
  '章节编号': '515',
  '编制环节用途': '指导投标人如何填写和提交投标函，明确投标函中的各项承诺和责任',
  '解读环节用途': '帮助招标人理解投标函的格式和内容要求，确保投标文件的完整性和合规性',
  '附件类型': '模板'}]


In [26]:
print(result)

[{'章节编号': '515', '章节标题': '附件 1 投标函', '附件类型': '模板', '摘要': '投标函模板，包含投标文件提交、保证金、合同责任等关键信息', '内容包括': '投标文件提交、投标保证金、合同责任、投标有效期、撤回投标的后果、提供额外数据或资料的承诺、正式通讯地址', '解读环节用途': '帮助招标人理解投标函的格式和内容要求，确保投标文件的完整性和合规性', '编制环节用途': '指导投标人如何填写和提交投标函，明确投标函中的各项承诺和责任', '关键行动': [{'行动名称': '提交投标文件', '时间': '投标截止日期前', '地点': '招标人指定地点', '内容': '提交投标函正本一份、副本若干份及电子版'}, {'行动名称': '支付投标保证金', '时间': '投标截止日期前', '地点': '招标人指定账户', '内容': '支付投标保证金，金额为人民币若干元'}, {'行动名称': '履行合同责任', '时间': '中标后', '地点': '项目实施地点', '内容': '按照招标文件规定履行合同责任和义务'}, {'行动名称': '撤回投标', '时间': '开标后投标有效期内', '地点': '招标人指定地点', '内容': '在投标有效期内撤回投标，投标保证金将被没收'}, {'行动名称': '提供额外数据或资料', '时间': '招标人要求时', '地点': '招标人指定地点', '内容': '提供与投标有关的一切数据或资料'}]}]


In [27]:
results = result

In [28]:
for result in results:
    print(f"章节编号: {result['章节编号']}" )
    print(f"章节标题: {result['章节标题']}")
    print(f"附件类型: {result['附件类型']}")
    print(f"摘要: {result['摘要']}")
    print(f"内容包括: {result['内容包括']}")
    print(f"解读环节用途: {result['解读环节用途']}")
    print(f"编制环节用途: {result['编制环节用途']}")
    print(f"关键行动: {result['关键行动']}")
    for node in doc_structure.doc_nodes:
        if node.node_id == int(result['章节编号']):
            node.appendix_type = result['附件类型']
            node.summary = result['摘要']
            node.content_include = result['内容包括']
            node.usage_in_interpretation = result['解读环节用途']
            node.usage_in_preparation = result['编制环节用途']
            node.key_actions = result['关键行动']


章节编号: 515
章节标题: 附件 1 投标函
附件类型: 模板
摘要: 投标函模板，包含投标文件提交、保证金、合同责任等关键信息
内容包括: 投标文件提交、投标保证金、合同责任、投标有效期、撤回投标的后果、提供额外数据或资料的承诺、正式通讯地址
解读环节用途: 帮助招标人理解投标函的格式和内容要求，确保投标文件的完整性和合规性
编制环节用途: 指导投标人如何填写和提交投标函，明确投标函中的各项承诺和责任
关键行动: [{'行动名称': '提交投标文件', '时间': '投标截止日期前', '地点': '招标人指定地点', '内容': '提交投标函正本一份、副本若干份及电子版'}, {'行动名称': '支付投标保证金', '时间': '投标截止日期前', '地点': '招标人指定账户', '内容': '支付投标保证金，金额为人民币若干元'}, {'行动名称': '履行合同责任', '时间': '中标后', '地点': '项目实施地点', '内容': '按照招标文件规定履行合同责任和义务'}, {'行动名称': '撤回投标', '时间': '开标后投标有效期内', '地点': '招标人指定地点', '内容': '在投标有效期内撤回投标，投标保证金将被没收'}, {'行动名称': '提供额外数据或资料', '时间': '招标人要求时', '地点': '招标人指定地点', '内容': '提供与投标有关的一切数据或资料'}]


In [29]:
for node in doc_structure.doc_nodes:
    if node.node_id == 515:
        print(node.element.content)
        print(node.appendix_type)
        print(node.summary)
        print(node.content_include)
        print(node.usage_in_interpretation)
        print(node.usage_in_preparation)
        print(node.key_actions)




附件 1 投标函
模板
投标函模板，包含投标文件提交、保证金、合同责任等关键信息
投标文件提交、投标保证金、合同责任、投标有效期、撤回投标的后果、提供额外数据或资料的承诺、正式通讯地址
帮助招标人理解投标函的格式和内容要求，确保投标文件的完整性和合规性
指导投标人如何填写和提交投标函，明确投标函中的各项承诺和责任
[{'行动名称': '提交投标文件', '时间': '投标截止日期前', '地点': '招标人指定地点', '内容': '提交投标函正本一份、副本若干份及电子版'}, {'行动名称': '支付投标保证金', '时间': '投标截止日期前', '地点': '招标人指定账户', '内容': '支付投标保证金，金额为人民币若干元'}, {'行动名称': '履行合同责任', '时间': '中标后', '地点': '项目实施地点', '内容': '按照招标文件规定履行合同责任和义务'}, {'行动名称': '撤回投标', '时间': '开标后投标有效期内', '地点': '招标人指定地点', '内容': '在投标有效期内撤回投标，投标保证金将被没收'}, {'行动名称': '提供额外数据或资料', '时间': '招标人要求时', '地点': '招标人指定地点', '内容': '提供与投标有关的一切数据或资料'}]


In [30]:
for result in results:
    print(f"\n处理章节编号: {result['章节编号']}")
    for node in doc_structure.doc_nodes:
        if node.node_id == int(result['章节编号']):
            print(f"找到匹配节点，开始赋值...")
            
            # 赋值前打印
            print("赋值前的属性值：")
            print(f"appendix_type: {node.appendix_type}")
            
            # 赋值
            node.appendix_type = result['附件类型']
            node.summary = result['摘要']
            node.content_include = result['内容包括']
            node.usage_in_interpretation = result['解读环节用途']
            node.usage_in_preparation = result['编制环节用途']
            node.key_actions = result['关键行动']
            
            # 赋值后立即打印
            print("赋值后的属性值：")
            print(f"appendix_type: {node.appendix_type}")
            print(f"summary: {node.summary}")
            print(f"content_include: {node.content_include}")
            print(f"usage_in_interpretation: {node.usage_in_interpretation}")
            print(f"usage_in_preparation: {node.usage_in_preparation}")
            print(f"key_actions: {node.key_actions}")
            
            # 确认属性是否可写
            print("\n属性是否可写：")
            print(f"hasattr(node, 'appendix_type'): {hasattr(node, 'appendix_type')}")
            print(f"type(node).__dict__: {type(node).__dict__}")


处理章节编号: 515
找到匹配节点，开始赋值...
赋值前的属性值：
appendix_type: 模板
赋值后的属性值：
appendix_type: 模板
summary: 投标函模板，包含投标文件提交、保证金、合同责任等关键信息
content_include: 投标文件提交、投标保证金、合同责任、投标有效期、撤回投标的后果、提供额外数据或资料的承诺、正式通讯地址
usage_in_interpretation: 帮助招标人理解投标函的格式和内容要求，确保投标文件的完整性和合规性
usage_in_preparation: 指导投标人如何填写和提交投标函，明确投标函中的各项承诺和责任
key_actions: [{'行动名称': '提交投标文件', '时间': '投标截止日期前', '地点': '招标人指定地点', '内容': '提交投标函正本一份、副本若干份及电子版'}, {'行动名称': '支付投标保证金', '时间': '投标截止日期前', '地点': '招标人指定账户', '内容': '支付投标保证金，金额为人民币若干元'}, {'行动名称': '履行合同责任', '时间': '中标后', '地点': '项目实施地点', '内容': '按照招标文件规定履行合同责任和义务'}, {'行动名称': '撤回投标', '时间': '开标后投标有效期内', '地点': '招标人指定地点', '内容': '在投标有效期内撤回投标，投标保证金将被没收'}, {'行动名称': '提供额外数据或资料', '时间': '招标人要求时', '地点': '招标人指定地点', '内容': '提供与投标有关的一切数据或资料'}]

属性是否可写：
hasattr(node, 'appendix_type'): True
type(node).__dict__: {'__module__': 'doc_structurer_package.doc_strucuturer._01_doc_node_creater', '__annotations__': {'node_id': <class 'int'>, 'element': <class 'doc_structurer_package.doc_strucuturer._01_doc_node_creater.SimpleEl